In [1]:
from sklearn.metrics import mutual_info_score
#from sklearn.feature_selection import chi2
from scipy.stats import chi2_contingency
from scipy.stats import chi2
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde, pearsonr
import seaborn as sns
import pandas as pd
from copy import copy

# Mathematical Underpinnings - Lab 5

Tests to verify hipoteses of independence (from Lab 4):

In [2]:
def indep_test_asymptotic(X, Y, stat):

    if stat == "mi":
        stat_value = 2 * len(X) * mutual_info_score(X, Y)

    if stat == "chi2":
        test_res = chi2_contingency(pd.crosstab(X, Y))
        stat_value = test_res.statistic

    df = (len(np.unique(X)) - 1) * (len(np.unique(Y)) - 1)

    p_value = 1 - chi2.cdf(stat_value, df=df)

    return stat_value, p_value


def indep_test_permutation(X, Y, B, stat="mi"):

    stat_value = mutual_info_score(X, Y)

    X = X.copy()
    Y = Y.copy()

    condition_p_value = 0
    for b in range(B):
        X_b = np.random.permutation(X)

        stat_value_b = mutual_info_score(X_b, Y)

        if stat_value <= stat_value_b:
            condition_p_value += 1

    p_value = (1 + condition_p_value) / (1 + B)

    return stat_value, p_value

## Task 1

In [3]:
# a function which computes CMI
def CMI(X, Y, Z):
    mi = 0

    for z in np.unique(Z):

        X_z = X[Z == z]
        Y_z = Y[Z == z]

        mi += mutual_info_score(X_z, Y_z) * len(X_z) / len(X)

    return mi

### a)

In [4]:
# CI test based on CMI and asymptotics
def cond_indep_test_asymptotic(X, Y, Z, stat="cmi"):

    X = X.copy()
    Y = Y.copy()
    Z = Z.copy()

    if stat == "cmi":

        stat_value = 2 * len(X) * CMI(X, Y, Z)

    df = (len(np.unique(X)) - 1) * (len(np.unique(Y)) - 1) * len(np.unique(Z))

    p_value = 1 - chi2.cdf(stat_value, df=df)

    return stat_value, p_value

### b)

In [5]:
# CI test based on CMI and permutations
def cond_indep_test_permutation(X, Y, Z, B = 1000):

    X = X.copy()
    Y = Y.copy()
    Z = Z.copy()
    
    stat_value = CMI(X, Y, Z)

    condition_p_value = 0
    for b in range(B):
        for z in np.unique(Z):

            # permute the values of X for each value of Z
            X[Z == z] = np.random.permutation(X[Z == z])

        stat_value_b = CMI(X, Y, Z)

        if stat_value <= stat_value_b:
            condition_p_value += 1
    p_value = (1 + condition_p_value)/(1 + B)
           
    return stat_value, p_value

### c)

**conditionaly independent**

* Let Z be uniform vector on {-1, 1}.
* Let X be discretized normal with mean = Z
* Let Y be discretized normal with mean = -Z

Then X is conditionally independent of Y through Z

In [6]:
seed = 123
N = 10000
rng = np.random.default_rng(seed)

Z = rng.binomial(1, 0.5, N)
X = rng.normal(0, 1, N) + Z/2
X = np.where(X>=0, 1, -1)
Y = rng.normal(0, 1, N) - Z/2
Y = np.where(Y>=0, 1, -1)

print("Asymptotic test ", cond_indep_test_asymptotic(X, Y, Z))
print("Permutation test ", cond_indep_test_permutation(X, Y, Z))


Asymptotic test  (1.2038518398983862, 0.5477556859798102)
Permutation test  (6.019259199491931e-05, 0.5484515484515484)


**conditionaly dependent**

* Let X be from discretized normal
* Let Z be from discretized normal
* Let Y = XOR(X, Z)

Then X is conditionally dependent with Z

In [7]:
X = rng.normal(0, 1, N)
X = np.where(X>=0, 1, 0)
Z = rng.normal(0, 1, N)
Z = np.where(Z>=0, 1, 0)

Y = np.logical_xor(X, Z).astype(int)

print("Asymptotic test ", cond_indep_test_asymptotic(X, Y, Z))
print("Permutation test ", cond_indep_test_permutation(X, Y, Z))

Asymptotic test  (13860.960777326525, 0.0)
Permutation test  (0.6930480388663263, 0.000999000999000999)


## Task 2

In [8]:
def sample_from_model1(n = 1000, seed = 123):
    rng = np.random.default_rng(seed) 
    Z_dash = rng.normal(0, 1, n)
    Z = np.where(Z_dash > 0, 1, -1)
    X_dash = rng.normal(0, 1, n) + Z/2
    Y_dash = rng.normal(0, 1, n) + Z/2
    X = np.where(X_dash > 0, 1, -1)
    Y = np.where(Y_dash > 0, 1, -1)
    return X, Y, Z

def sample_from_model2(n = 1000, seed = 123):
    rng = np.random.default_rng(seed)
    X_dash = rng.normal(0, 1, n)
    X = np.where(X_dash > 0, 1, -1)
    Z_dash = rng.normal(0, 1, n) + X/2
    Z = np.where(Z_dash > 0, 1, -1)
    Y_dash = rng.normal(0, 1, n) + Z/2
    Y = np.where(Y_dash > 0, 1, -1)

    return X, Y, Z

def sample_from_model3(n = 1000, seed = 123):
    rng = np.random.default_rng(seed)

    X_dash = rng.normal(0, 1, n)
    X = np.where(X_dash > 0, 1, -1)
    Y_dash = rng.normal(0, 1, n)
    Y = np.where(Y_dash > 0, 1, -1)
    Z_dash = rng.normal(0, 1, n) + X/2 + Y/2
    Z = np.where(Z_dash > 0, 1, -1)

    return X, Y, Z

### a)

answer:

* Model 1 : conditionally independent through Z, dependent

* Model 2 : conditionally independent through Z, dependent

* Model 3 : conditionally dependent through Z, independent

### b)

In [9]:
# MI (X, Y)
seed = 123
X_m1, Y_m1, Z_m1 = sample_from_model1(n = 10000, seed = seed)
X_m2, Y_m2, Z_m2 = sample_from_model2(n = 10000, seed = seed)
X_m3, Y_m3, Z_m3 = sample_from_model3(n = 10000, seed = seed)


print("Model 1: MI(X, Y)=", mutual_info_score(X_m1, Y_m1))
print("Model 2: MI(X, Y)=", mutual_info_score(X_m2, Y_m2))
print("Model 3: MI(X, Y)=", mutual_info_score(X_m3, Y_m3))

# CMI (X, Y|Z)
print("Model 1: CMI(X, Y|Z)=", CMI(X_m1, Y_m1, Z_m1))
print("Model 2: CMI(X, Y|Z)=", CMI(X_m2, Y_m2, Z_m2))
print("Model 3: CMI(X, Y|Z)=", CMI(X_m3, Y_m3, Z_m3))


Model 1: MI(X, Y)= 0.013517278650951647
Model 2: MI(X, Y)= 0.012469746895038436
Model 3: MI(X, Y)= 3.820875885457431e-05
Model 1: CMI(X, Y|Z)= 0.0001503073618825845
Model 2: CMI(X, Y|Z)= 8.719142638590844e-06
Model 3: CMI(X, Y|Z)= 0.008107906643733637


### c)

In [10]:
# permutation test for CMI

print("Model 1: permutation test CMI(X, Y|Z)=", cond_indep_test_permutation(X_m1, Y_m1, Z_m1, 1000))
print("Model 2: permutation test CMI(X, Y|Z)=", cond_indep_test_permutation(X_m2, Y_m2, Z_m2, 1000))
print("Model 3: permutation test CMI(X, Y|Z)=", cond_indep_test_permutation(X_m3, Y_m3, Z_m3, 1000))

# asymptotic test for CMI
print("Model 1: asymptotic test CMI(X, Y|Z)=", cond_indep_test_asymptotic(X_m1, Y_m1, Z_m1))
print("Model 2: asymptotic test CMI(X, Y|Z)=", cond_indep_test_asymptotic(X_m2, Y_m2, Z_m2))
print("Model 3: asymptotic test CMI(X, Y|Z)=", cond_indep_test_asymptotic(X_m3, Y_m3, Z_m3))

Model 1: permutation test CMI(X, Y|Z)= (0.0001503073618825845, 0.2137862137862138)
Model 2: permutation test CMI(X, Y|Z)= (8.719142638590844e-06, 0.922077922077922)
Model 3: permutation test CMI(X, Y|Z)= (0.008107906643733637, 0.000999000999000999)
Model 1: asymptotic test CMI(X, Y|Z)= (3.00614723765169, 0.2224453959786935)
Model 2: asymptotic test CMI(X, Y|Z)= (0.17438285277181687, 0.9165016362440044)
Model 3: asymptotic test CMI(X, Y|Z)= (162.15813287467276, 0.0)


Both tests indicate, that X is **conditionally independent** of Y through Z in Models 1 and 2. (there is not enough evidence to disprove the null hypothesis of conditional independence).

Both tests indicate, that X is **not conditionally independent** of Y through Z in Model 3 

In [11]:
# test for independence
print("Model 1: permutation test MI(X, Y)=", indep_test_permutation(X_m1, Y_m1, 1000))
print("Model 2: permutation test MI(X, Y)=", indep_test_permutation(X_m2, Y_m2, 1000))
print("Model 3: permutation test MI(X, Y)=", indep_test_permutation(X_m3, Y_m3, 1000))

print("Model 1: asymptotic test MI(X, Y)=", indep_test_asymptotic(X_m1, Y_m1, "mi"))
print("Model 2: asymptotic test MI(X, Y)=", indep_test_asymptotic(X_m2, Y_m2, "mi"))
print("Model 3: asymptotic test MI(X, Y)=", indep_test_asymptotic(X_m3, Y_m3, "mi"))

print("Model 1: asymptotic test (pearson)=", indep_test_asymptotic(X_m1, Y_m1, "chi2"))
print("Model 2: asymptotic test (pearson)=", indep_test_asymptotic(X_m2, Y_m2, "chi2"))
print("Model 3: asymptotic test (pearson)=", indep_test_asymptotic(X_m3, Y_m3, "chi2"))

Model 1: permutation test MI(X, Y)= (0.013517278650951647, 0.000999000999000999)
Model 2: permutation test MI(X, Y)= (0.012469746895038436, 0.000999000999000999)
Model 3: permutation test MI(X, Y)= (3.820875885457431e-05, 0.4025974025974026)
Model 1: asymptotic test MI(X, Y)= (270.34557301903294, 0.0)
Model 2: asymptotic test MI(X, Y)= (249.39493790076872, 0.0)
Model 3: asymptotic test MI(X, Y)= (0.7641751770914862, 0.3820250617774704)
Model 1: asymptotic test (pearson)= (268.4781764616364, 0.0)
Model 2: asymptotic test (pearson)= (247.73726157085164, 0.0)
Model 3: asymptotic test (pearson)= (0.7295927080792952, 0.393015239510876)


All tests indicate, that X is **not independent** of Y in Models 1 and 2.

All tests indicate, that X is **independent** of Y in Model 3 (there is no evidence to disprove the null hypothesis of independence).